In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
A = open("/content/drive/My Drive/sherlock.txt")

Preprocessing
1. Case Unification
2. Special Charater Removal
3. Stemming
4. Lemmatization

In [20]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
import re

Q = []

from nltk.stem import PorterStemmer,WordNetLemmatizer

wn = WordNetLemmatizer()
ps = PorterStemmer()

for i in A.read().split(" "):
  i = ps.stem(i)
  i = wn.lemmatize(i)
  i = i.upper()
  i = re.sub("[^A-Z]","",i)
  Q.append(i)

Q = Q[0:5000]

Create Unique words

In [22]:
len(Q)

5000

In [23]:
import numpy as np
unq = np.unique(Q)

In [24]:
unq[10:15]

array(['ACCOMPLISH', 'ACCOUNT', 'ACCUSTOM', 'ACHARACT', 'ACHURCH'],
      dtype='<U21')

Preparing input and output

In [25]:
inp = []
op = []
for i in range(4,len(Q),1):
  w1 = Q[i-4]
  w2 = Q[i-3]
  w3 = Q[i-2]
  w4 = Q[i-1]
  sentence = str(w1)+" "+str(w2)+" "+str(w3)+" "+str(w4)
  inp.append(sentence)
  op.append(Q[i])

In [26]:
import pandas as pd
A = pd.DataFrame([inp,op]).T

In [27]:
A.columns = ["X","Y"]

Find out dimension of input

In [28]:
nlines = len(A.X)
num_uniques = len(unq)
sequence = 4

Create a blank array containing all zeroes with required dimensions

In [29]:
X = np.zeros((nlines,num_uniques,sequence),dtype=bool)
Y = np.zeros((nlines,num_uniques),dtype=bool)

Create a list of unique words which can be used for returning index values of specific words

In [30]:
U1 = list(unq)
U1.index("PROJECT")

1181

Update value in the cube|Tensor|Array for X

In [31]:
for i,j in enumerate(A.X):
    words_in_a_line = j.split(" ")[0:4]
    for p,q in enumerate(words_in_a_line):
        X[i,U1.index(q),p] = 1

Update value in the cube|Tensor|Array for Y

In [32]:
for i,j in enumerate(A.Y):
    Y[i,U1.index(j)]=1

In [33]:
Y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [34]:
Y[1,U1.index("SHERLOCK")]

True

Create a Neaural Network

In [35]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Activation
from keras.activations import softmax

nn = Sequential()
nn.add(LSTM(128,input_shape=(1726,4)))
nn.add(Dense(len(U1)))
nn.add(Activation('softmax'))

In [36]:
nn.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model = nn.fit(X,Y,epochs=2)

Epoch 1/2
157/157 [==============================] - 225s 1s/step - loss: 6.7442 - accuracy: 0.0436
Epoch 2/2
157/157 [==============================] - 224s 1s/step - loss: 6.3140 - accuracy: 0.0494


In [37]:
X.shape

(4996, 1726, 4)

Get input from user

In [38]:
input_X = np.zeros((1,1726,4),dtype=bool)
inp = input("Enter text: ")

Enter text: The adventures of sherlock


Prepare user input same as train data

In [39]:
I = []
for i in inp.split(" "):
    i = ps.stem(i)
    i = wn.lemmatize(i)
    i = i.upper()
    i = re.sub("[^A-Z]","",i)
    I.append(i)

In [40]:
I

['THE', 'ADVENTUR', 'OF', 'SHERLOCK']

In [41]:
for p,q in enumerate(I):
    input_X[0,U1.index(q),p] = 1

In [42]:
input_X

array([[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        ...,
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]]])

In [43]:
U1.index("SHERLOCK")

1323

In [44]:
input_X[0,1323,3]

True

Predicting what will be next words

In [45]:
OP_NN = nn.predict(input_X)

Making Dataframe of high probabilty next words

In [46]:
OP_DF = pd.DataFrame([U1,OP_NN[0]]).T

In [47]:
OP_DF.columns = ["word","prob"]

Printing the first high probabilty word

In [48]:
print("Next possible words are as follows: ")
for i in OP_DF.sort_values(by="prob",ascending=False)[["word"]][0:5].values:
    print(i[0])

Next possible words are as follows: 
THE
A
AND
OF
TO
